In [1]:
!mkdir -p data/roadmap && cd data/roadmap

# Histone modification bigWig files for cell type E116 (GM12878)
!wget https://egg2.wustl.edu/roadmap/data/byFileType/signal/consolidated/macs2signal/foldChange/E116-H3K4me3.fc.signal.bigwig
!wget https://egg2.wustl.edu/roadmap/data/byFileType/signal/consolidated/macs2signal/foldChange/E116-H3K27ac.fc.signal.bigwig
!wget https://egg2.wustl.edu/roadmap/data/byFileType/signal/consolidated/macs2signal/foldChange/E116-H3K36me3.fc.signal.bigwig

# Gene expression matrix and metadata
!wget https://egg2.wustl.edu/roadmap/data/byDataType/rna/expression/57epigenomes.RPKM.pc.gz
!wget https://egg2.wustl.edu/roadmap/data/byDataType/rna/expression/EG.name.txt
!wget https://egg2.wustl.edu/roadmap/data/byDataType/rna/expression/Ensembl_v65.Gencode_v10.ENSG.gene_info

# Unzip the expression file
!gunzip -f 57epigenomes.RPKM.pc.gz



--2025-07-31 18:47:09--  https://egg2.wustl.edu/roadmap/data/byFileType/signal/consolidated/macs2signal/foldChange/E116-H3K4me3.fc.signal.bigwig
Resolving egg2.wustl.edu (egg2.wustl.edu)... 128.252.187.85
Connecting to egg2.wustl.edu (egg2.wustl.edu)|128.252.187.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 479312792 (457M)
Saving to: ‘E116-H3K4me3.fc.signal.bigwig’

E116-H3K4me3.fc.sig 100%[===================>] 457.11M  1.08MB/s    in 6m 59s  

2025-07-31 18:54:08 (1.09 MB/s) - ‘E116-H3K4me3.fc.signal.bigwig’ saved [479312792/479312792]

--2025-07-31 18:54:08--  https://egg2.wustl.edu/roadmap/data/byFileType/signal/consolidated/macs2signal/foldChange/E116-H3K27ac.fc.signal.bigwig
Resolving egg2.wustl.edu (egg2.wustl.edu)... 128.252.187.85
Connecting to egg2.wustl.edu (egg2.wustl.edu)|128.252.187.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293665780 (280M)
Saving to: ‘E116-H3K27ac.fc.signal.bigwig’

E116-H3K27ac.fc.sig 10

In [2]:
!pip install pyBigWig


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 kB 7.9 MB/s eta 0:00:00


In [8]:
# ⚙️ Setup: Mount Google Drive to save final dataset
from google.colab import drive
drive.mount('/content/drive')
# 🔢 Expression data
import pandas as pd
import numpy as np

eg_map = pd.read_csv("EG.name.txt", sep="\t", header=None, names=["EID", "Tissue"])
expr_df = pd.read_csv("57epigenomes.RPKM.pc", sep=r"\s+", header=None)

gene_ids = expr_df.iloc[:, 0].astype(str).str.split(".").str[0]
expr_values = expr_df.iloc[:, 1:]
expr_values.index = gene_ids

e116_idx = eg_map[eg_map["EID"] == "E116"].index[0]
e116_col = pd.to_numeric(expr_values.iloc[:, e116_idx], errors='coerce').dropna()
e116_log_expr = np.log2(e116_col + 1)

print("✅ Loaded expression values:", e116_log_expr.shape)

# 🧬 Gene annotation
gene_info = pd.read_csv(
    "Ensembl_v65.Gencode_v10.ENSG.gene_info",
    sep=r"\s+",
    header=None,
    names=[
        "gene_id", "chr", "start", "end", "strand", "biotype", "symbol", "description"
    ],
    usecols=[0, 1, 2, 3, 4, 6]
)

gene_info["gene_id"] = gene_info["gene_id"].astype(str).str.split(".").str[0]
gene_info["strand"] = gene_info["strand"].map({1: "+", -1: "-"})
gene_info["tss"] = np.where(gene_info["strand"] == "+", gene_info["start"], gene_info["end"])

valid_chr = [str(i) for i in range(1, 23)] + ["X", "Y"]
gene_info = gene_info[gene_info["chr"].isin(valid_chr)]
gene_info = gene_info[gene_info["gene_id"].isin(e116_log_expr.index)]
print(f"✅ Total matched genes: {len(gene_info)}")

# 📈 Histone signals extraction
import pyBigWig
from tqdm import tqdm

bw_k4me3 = pyBigWig.open("E116-H3K4me3.fc.signal.bigwig")
bw_k27ac = pyBigWig.open("E116-H3K27ac.fc.signal.bigwig")
bw_k36me3 = pyBigWig.open("E116-H3K36me3.fc.signal.bigwig")
bw_list = [bw_k4me3, bw_k27ac, bw_k36me3]

def extract_histone_signal(chrom, tss, bins=100, flank=5000):
    bin_size = 2 * flank // bins
    starts = [tss - flank + i * bin_size for i in range(bins)]
    ends = [s + bin_size for s in starts]

    features = []
    for bw in bw_list:
        row = []
        for start, end in zip(starts, ends):
            try:
                vals = bw.values(f"chr{chrom}", max(0, start), end, numpy=True)
                row.append(np.nanmean(vals))
            except:
                row.append(0.0)
        features.append(row)
    return np.array(features).T  # shape [100, 3]

# 🔁 Extract across all genes
X_all, y_all, gene_ids_all = [], [], []
print("⏳ Extracting histone features...")

for _, row in tqdm(gene_info.iterrows(), total=len(gene_info)):
    gene_id = row["gene_id"]
    chrom = row["chr"]
    tss = int(row["tss"])

    try:
        signal = extract_histone_signal(chrom, tss)
        if signal.shape == (100, 3):
            X_all.append(signal)
            y_all.append(e116_log_expr[gene_id])
            gene_ids_all.append(gene_id)
    except:
        continue

X = np.stack(X_all)
y = np.array(y_all)
gene_ids = np.array(gene_ids_all)

print(f"✅ Final shapes: X={X.shape}, y={y.shape}")

# 💾 Save to Google Drive /content/drive/MyDrive/Gene_Expression_Prediction_Histone_Marks/Phase_1/Dataset/expression_chr22.npz
np.savez("/content/drive/MyDrive/Gene_Expression_Prediction_Histone_Marks/Phase_1/Dataset/expression_E116_all.npz", X=X, y=y, gene_ids=gene_ids)
print("📁 Saved to /content/drive/MyDrive/Gene_Expression_Prediction_Histone_Marks/Phase_1/Dataset/expression_E116_all.npz")


Mounted at /content/drive


/tmp/ipython-input-1939934608.py:9: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  expr_df = pd.read_csv("57epigenomes.RPKM.pc", sep=r"\s+", header=None)


✅ Loaded expression values: (19795,)
✅ Total matched genes: 19802
⏳ Extracting histone features...


 97%|█████████▋| 19186/19802 [06:19<00:11, 52.49it/s]/tmp/ipython-input-1939934608.py:61: RuntimeWarning: Mean of empty slice
  row.append(np.nanmean(vals))
100%|██████████| 19802/19802 [06:31<00:00, 50.63it/s]


✅ Final shapes: X=(19802, 100, 3), y=(19802,)
📁 Saved to /content/drive/MyDrive/Gene_Expression_Prediction_Histone_Marks/Phase_1/Dataset/expression_E116_all.npz
